In [1]:
from __future__ import print_function
import argparse

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

import os
import time

from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_n

In [2]:
batch_size = 128
test_batch_size = 1000

In [3]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./mnist_data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, 
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./mnist_data', train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size,
    shuffle=True
)

### Model fit

In [4]:
class Teacher(nn.Module):
    def __init__(self):
        super(Teacher, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 1200)
        self.fc2 = nn.Linear(1200, 1200)
        self.fc3 = nn.Linear(1200, 600)
        self.fc4 = nn.Linear(600, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.8, training=self.training)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.8, training=self.training)
        x = F.relu(self.fc3(x))
        x = F.dropout(x, p=0.8, training=self.training)
        x = self.fc4(x)
        return x

In [5]:
teacher = Teacher()

In [6]:
def train(model, optimizer, epochs, eval_per_epochs, train_loader, test_loader):
    for epoch in range(1, epochs + 1):
        model.train()

        epoch_loss = 0
        for batch_idx, (data, target) in tqdm(enumerate(train_loader), total=len(train_loader)):
            optimizer.zero_grad()

            data, target = Variable(data), Variable(target)
            output = model(data)

            loss = F.cross_entropy(output, target)
            loss.backward()
            epoch_loss += loss.item()

            optimizer.step()

        epoch_loss /= len(train_loader)
        print(f'Epoch: {epoch}, epoch loss: {epoch_loss:.6f}')

        if epoch % eval_per_epochs == 0:
            model.eval()

            test_loss = 0
            correct = 0
            for data, target in test_loader:
                data, target = Variable(data, volatile=True), Variable(target)
                output = model(data)
                test_loss += F.cross_entropy(output, target).item()

                pred = output.data.max(1, keepdim=True)[1]
                correct += pred.eq(target.data.view_as(pred)).sum()

            print(f'Test loss: {test_loss / len(test_loader):.4f}, '
                  f'accuracy: {correct.item() / len(test_loader.dataset):.3f}')

In [7]:
epochs = 10
lr = 0.01
momentum = 0.5

optimizer = optim.SGD(teacher.parameters(), lr=lr, momentum=momentum)

In [8]:
epochs = 50
eval_per_epochs = 10

train(teacher, optimizer, epochs, eval_per_epochs, train_loader, test_loader)

  0%|          | 2/469 [00:00<00:24, 18.68it/s]

Epoch: 1, epoch loss: 1.903513


  0%|          | 2/469 [00:00<00:28, 16.66it/s]

Epoch: 2, epoch loss: 0.933519


  0%|          | 2/469 [00:00<00:33, 14.14it/s]

...

Epoch: 48, epoch loss: 0.104591


  1%|          | 3/469 [00:00<00:16, 27.42it/s]

Epoch: 49, epoch loss: 0.105738


100%|██████████| 469/469 [00:18<00:00, 24.74it/s]


Epoch: 50, epoch loss: 0.100993
Test loss: 0.0757, accuracy: 0.980


In [9]:
torch.save(teacher.state_dict(), 'models/teacher.pth')

### Distillation

In [10]:
class Student(nn.Module):
    def __init__(self):
        super(Student, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 300)
        self.fc2 = nn.Linear(300, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [11]:
student = Student()

In [12]:
def distillation_loss(y, labels, teacher_scores, T=20, alpha=0.7):
    kldivloss = nn.KLDivLoss()(F.log_softmax(y/T), F.softmax(teacher_scores/T))
    temp_impact = (T*T * 2.0 * alpha)
    cross_entropy = F.cross_entropy(y, labels) * (1. - alpha)
    loss = kldivloss * temp_impact + cross_entropy
    return loss

In [13]:
def train(model, teacher_model, loss_fn, optimizer, epochs, eval_per_epochs):
    teacher_model.eval()

    epoch_loss = 0
    for epoch in range(1, epochs + 1):
        model.train()

        for batch_idx, (data, target) in tqdm(enumerate(train_loader), total=len(train_loader)):
            optimizer.zero_grad()

            data, target = Variable(data), Variable(target)
            output = model(data)
            teacher_output = Variable(teacher_model(data), requires_grad=False)

            loss = loss_fn(output, target, teacher_output)
            loss.backward()
            epoch_loss += loss.item()

            optimizer.step()

        epoch_loss /= len(train_loader)
        print(f'Epoch: {epoch}, epoch distillation loss: {epoch_loss:.6f}')

        if epoch % eval_per_epochs == 0:
            model.eval()

            test_loss = 0
            correct = 0
            for data, target in test_loader:
                data, target = Variable(data, volatile=True), Variable(target)
                output = model(data)
                test_loss += F.cross_entropy(output, target).item()

                pred = output.data.max(1, keepdim=True)[1]
                correct += pred.eq(target.data.view_as(pred)).sum()

            print(f'Student test loss: {test_loss / len(test_loader):.4f}, '
                  f'student accuracy: {correct.item() / len(test_loader.dataset):.3f}')

In [14]:
epochs = 10
lr = 0.01
momentum = 0.9

optimizer = optim.SGD(student.parameters(), lr=lr, momentum=momentum)

In [ ]:
epochs = 50
eval_per_epochs = 10

train(student, teacher, distillation_loss, optimizer, epochs, eval_per_epochs)

  0%|          | 0/469 [00:00<?, ?it/s]/data/artm10/lib/python3.7/site-packages/ipykernel/__main__.py:2: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app
/data/artm10/lib/python3.7/site-packages/ipykernel/__main__.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app
/data/artm10/lib/python3.7/site-packages/torch/nn/functional.py:1946: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."
  1%|          | 3/469 [00:00<00:15, 29.34it/s]

Epoch: 1, epoch distillation loss: 1.001934


  1%|          | 5/469 [00:00<00:10, 45.25it/s]

Epoch: 2, epoch distillation loss: 0.239985


  1%|          | 5/469 [00:00<00:10, 45.14it/s]

...

  1%|          | 3/469 [00:00<00:17, 27.38it/s]

Epoch: 12, epoch distillation loss: 0.047963


  1%|          | 5/469 [00:00<00:10, 45.10it/s]

Epoch: 13, epoch distillation loss: 0.045596


 78%|███████▊  | 367/469 [00:09<00:02, 45.95it/s]

In [ ]:
torch.save(student.state_dict(), 'models/student.pth.tar')